# Purpose:
- Build dataframe of whisker features per trial
- To calculate angle discrimination using whisker features, changes of sensory input across training, etc.
- Per trial, take mean of whisker features across all touches, except for touch count
- Save for pre-answer touch, post-answer touch, all touch
    - pre-answer touch: touches that ended before the answer time
    - post-answer touch: touches that started after the answer time

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import glob
from tqdm.notebook import tqdm
w_dir = Path(r'E:\TPM\JK\WhiskerVideo')
b_dir = Path(r'E:\TPM\JK\SoloData')
results_dir = Path(r'E:\TPM\JK\h5\results')
mice =          [25,    27,   30,   36,     37,     38,     39,     41,     52,     53,     54,     56]


In [3]:
mouse = mice[0]
session_folder_list = glob.glob(str(w_dir / f'JK{mouse:03d}S*'))
session_folder = session_folder_list[0]
session_name = session_folder.split('\\')[-1]
whisker_pkl_fn = w_dir / session_name / f'{session_name}_whisker.pkl'
whisker_df = pd.read_pickle(whisker_pkl_fn)

b_fn = b_dir / f'JK{mouse:03d}' / f'{session_name}_behavior.pkl'
b_df = pd.read_pickle(b_fn)

In [14]:
session_name

'JK025S01'

In [6]:
whisker_df.columns

Index(['amplitude', 'arcLength', 'kappaH', 'kappaV', 'midpoint', 'phase',
       'phi', 'poleAngle', 'poleDistance', 'poleMovingFrames', 'polePosition',
       'poleUpFrames', 'protractionSlide', 'protractionSlideByWhisking',
       'protractionTFchunks', 'protractionTFchunksByWhisking',
       'protractionTouchDuration', 'protractionTouchDurationByWhisking',
       'retractionSlide', 'retractionSlideByWhisking', 'retractionTFchunks',
       'retractionTFchunksByWhisking', 'retractionTouchDuration',
       'retractionTouchDurationByWhisking', 'theta', 'whisker_time',
       'whiskingStartFrames', 'mouse_name', 'session_name'],
      dtype='object')

In [16]:
b_df.columns

Index(['answerLickTime', 'answerPeriodTime', 'beamBreakTimesLeft',
       'beamBreakTimesRight', 'choice', 'drinkingTime',
       'extraITIOnErrorSetting', 'motorApPosition', 'motorDistance',
       'poleDownOnsetTime', 'poleUpOnsetTime', 'rewardTimeLeft',
       'rewardTimeRight', 'samplingPeriodTime', 'servoAngle', 'trialType',
       'mouse_name', 'session_name', 'session_type', 'task_target',
       'distractor'],
      dtype='object')

In [105]:
def make_all_list_arrays(arrays):
    arrays = [np.array([ar]) if len(ar.shape) == 0 else ar for ar in arrays]
    return arrays


trialNums = whisker_df.index.values
ti = 0
trialNum = trialNums[ti]
trial_series = whisker_df.loc[trialNum]
touch_chunks = trial_series.protractionTFchunksByWhisking
touch_chunks = make_all_list_arrays(touch_chunks)
touch_count = len(touch_chunks)
touch_onset_frames = [int(chunk[0]) for chunk in touch_chunks]
touch_offset_frames = [int(chunk[-1]) for chunk in touch_chunks]
touch_onset_times = trial_series.whisker_time[touch_onset_frames]
touch_offset_times = trial_series.whisker_time[touch_offset_frames]

trial_b_df = b_df.loc[trialNum]
answer_lick_time = trial_b_df.answerLickTime

touch_duration = [off_t - on_t for (on_t, off_t) in zip(touch_onset_times, touch_offset_times)]
protractionSlideByWhisking = make_all_list_arrays(trial_series.protractionSlideByWhisking)
slide_distance = [max(sd) for sd in protractionSlideByWhisking]

# onset_features
onset_features = ['theta', 'phi', 'kappaH', 'kappaV', 'arcLength']
onset_values = {}
for feature in onset_features:
    onset_values[feature+'_onset'] = [trial_series.loc[feature][onset_frame] for onset_frame in touch_onset_frames]
# during_features
during_values = {}
theta_during = [trial_series.loc['theta'][onset_frame:offset_frame+1] for (onset_frame, offset_frame) in zip(touch_onset_frames, touch_offset_frames)]
theta_during_max = [np.amax(values-values[0]) for values in theta_during]
kappaH_during = [trial_series.loc['kappaH'][onset_frame:offset_frame+1] for (onset_frame, offset_frame) in zip(touch_onset_frames, touch_offset_frames)]
kappaH_during_min = [np.amin(values-values[0]) for values in kappaH_during] # for protraction touch, kappaH should decrease
during_values['delta_theta'] = theta_during_max
during_values['delta_kappaH'] = kappaH_during_min
during_features_sign = ['phi', 'kappaV']  # delta_phi and delta_kappaV can have both signs
for feature in during_features_sign:
    temp_values = [trial_series.loc[feature][onset_frame:offset_frame+1] for (onset_frame, offset_frame) in zip(touch_onset_frames, touch_offset_frames)]
    during_values['delta_' + feature] = [np.amax(np.abs(values - values[0])) * \
                              np.sign(values[np.argmax(np.abs(values-values[0]))] - values[0]) \
                              for values in temp_values]
touch_values = {'trialNum': [trialNum]*touch_count,
                'touch_onset_time': touch_onset_times,
                'touch_offset_time': touch_offset_times,
                'answer_lick_time': [answer_lick_time]*touch_count,
                'touch_duration': touch_duration,
                'slide_distance': slide_distance,
                }
touch_values.update(onset_values)
touch_values.update(during_values)
touch_features = pd.DataFrame(touch_values)


In [14]:
def get_touch_feature_df_each_trial(w_trial, b_trial):
    assert w_trial.name == b_trial.name
    trialNum = w_trial.name    
    touch_chunks = w_trial.protractionTFchunksByWhisking
    touch_chunks = make_all_list_arrays(touch_chunks)
    touch_count = len(touch_chunks)
    if touch_count == 0:
        return None
    touch_onset_frames = [int(chunk[0]) for chunk in touch_chunks]
    touch_offset_frames = [int(chunk[-1]) for chunk in touch_chunks]
    touch_onset_times = w_trial.whisker_time[touch_onset_frames]
    touch_offset_times = w_trial.whisker_time[touch_offset_frames]

    answer_lick_time = b_trial.answerLickTime

    touch_duration = [off_t - on_t for (on_t, off_t) in zip(touch_onset_times, touch_offset_times)]
    protractionSlideByWhisking = make_all_list_arrays(w_trial.protractionSlideByWhisking)
    slide_distance = [max(sd) for sd in protractionSlideByWhisking]
    # onset_features
    onset_features = ['theta', 'phi', 'kappaH', 'kappaV', 'arcLength']
    onset_values = {}
    for feature in onset_features:
        onset_values[feature+'_onset'] = [w_trial.loc[feature][onset_frame] for onset_frame in touch_onset_frames]
    # during_features
    during_values = {}
    theta_during = [w_trial.loc['theta'][onset_frame:offset_frame+1]\
                    for (onset_frame, offset_frame) in zip(touch_onset_frames, touch_offset_frames)]
    theta_during_max = [np.amax(values-values[0]) for values in theta_during]
    kappaH_during = [w_trial.loc['kappaH'][onset_frame:offset_frame+1]\
                     for (onset_frame, offset_frame) in zip(touch_onset_frames, touch_offset_frames)]
    kappaH_during_min = [np.amin(values-values[0]) for values in kappaH_during] # for protraction touch, kappaH should decrease
    during_values['delta_theta'] = theta_during_max
    during_values['delta_kappaH'] = kappaH_during_min
    during_features_sign = ['phi', 'kappaV']  # delta_phi and delta_kappaV can have both signs
    for feature in during_features_sign:
        temp_values = [w_trial.loc[feature][onset_frame:offset_frame+1]\
                       for (onset_frame, offset_frame) in zip(touch_onset_frames, touch_offset_frames)]
        during_values['delta_' + feature] = [np.amax(np.abs(values - values[0])) * \
                                np.sign(values[np.argmax(np.abs(values-values[0]))] - values[0]) \
                                for values in temp_values]
    touch_values = {'trialNum': [trialNum]*touch_count,
                    'touch_onset_time': touch_onset_times,
                    'touch_offset_time': touch_offset_times,
                    'answer_lick_time': [answer_lick_time]*touch_count,
                    'touch_duration': touch_duration,
                    'slide_distance': slide_distance,
                    }
    touch_values.update(onset_values)
    touch_values.update(during_values)
    touch_features = pd.DataFrame(touch_values)
    return touch_features


def make_all_list_arrays(arrays):
    arrays = [np.array([ar]) if len(ar.shape) == 0 else ar for ar in arrays]
    return arrays

In [5]:
trialNums = whisker_df.index.values
ti = 0
trialNum = trialNums[ti]
w_trial = whisker_df.loc[trialNum]
b_trial = b_df.loc[trialNum]
touch_features = get_touch_feature_df_each_trial(w_trial, b_trial)

In [111]:
ti = 1
trialNum = trialNums[ti]
w_trial = whisker_df.loc[trialNum]
b_trial = b_df.loc[trialNum]
temp_features = get_touch_feature_df_each_trial(w_trial, b_trial)

In [112]:
touch_features = pd.concat([touch_features, temp_features], ignore_index=True)
touch_features

,trial_num,touch_onset_time,touch_offset_time,answer_lick_time,touch_duration,slide_distance,theta_onset,phi_onset,kappaH_onset,kappaV_onset,arcLength_onset,delta_theta,delta_kappaH,delta_phi,delta_kappaV
0,2,1.641820,1.645033,2.621974,0.003213,0.051033,0.087511,-3.665535,0.044734,-0.051098,5.261802,0.000000,0.000000,-0.263482,-0.006014
1,2,1.677162,1.683588,2.621974,0.006426,0.053029,3.056956,-3.131719,0.042326,-0.050991,5.250117,0.000000,0.000000,1.940506,-0.008177
2,2,1.702866,1.712505,2.621974,0.009639,0.272299,3.815146,-2.286458,0.028474,-0.051796,5.149981,0.563682,-0.007285,2.071100,0.003267
3,2,1.776764,1.776764,2.621974,0.000000,0.000000,-3.952790,-5.403471,0.037733,-0.050567,5.287988,0.000000,0.000000,0.000000,0.000000
4,2,1.818532,1.831384,2.621974,0.012852,0.490961,-4.444834,-4.099098,0.045884,-0.055573,5.354224,1.942225,-0.011300,0.848516,0.006600
5,2,1.911708,1.921347,2.621974,0.009639,0.115360,-2.156975,-2.516048,0.039459,-0.054350,5.287414,1.447824,-0.019255,0.608456,0.004487
6,2,1.956689,1.966328,2.621974,0.009639,0.388997,1.821158,5.980232,0.044408,-0.067287,5.343034,2.284215,-0.003683,1.747998,0.007976
7,2,1.995245,2.014523,2.621974,0.019278,0.430279,5.624179,8.830789,0.031355,-0.055614,5.391150,2.780216,-0.016358,2.982291,-0.007661
8,2,2.053078,2.062717,2.621974,0.009639,0.038520,4.597950,9.436460,0.014841,-0.057001,5.334294,0.582080,-0.004064,-0.184975,-0.002643
9,2,2.104485,2.143041,2.621974,0.038555,0.816652,2.116042,11.184362,0.037175,-0.053467,5.441736,7.373709,-0.019084,3.783348,-0.009292


In [113]:
touch_features = {}
len(touch_features)

0

In [9]:
trialNums = whisker_df.index.values
touch_features = {}
for trialNum in trialNums:
    if len(touch_features) == 0:
        touch_features = get_touch_feature_df_each_trial(whisker_df.loc[trialNum], b_df.loc[trialNum])
    else:
        temp_features = get_touch_feature_df_each_trial(whisker_df.loc[trialNum], b_df.loc[trialNum])
        if temp_features is not None:
            touch_features = pd.concat([touch_features, temp_features], ignore_index=True)

In [117]:
len(touch_features)

6914

In [15]:
def get_session_touch_whisker_features(whisker_df, b_df):
    trialNums = whisker_df.index.values
    touch_features = {}
    for trialNum in trialNums:
        temp_features = get_touch_feature_df_each_trial(whisker_df.loc[trialNum], b_df.loc[trialNum])
        if temp_features is not None:
            if len(touch_features) == 0:
                touch_features = temp_features
            else:
                touch_features = pd.concat([touch_features, temp_features], ignore_index=True)
    return touch_features

In [16]:
# Takes about 17 min to finish all sessions from all 12 mice
save_dir = results_dir / 'touch_whisker_features'
save_dir.mkdir(exist_ok=True)
for mouse in tqdm(mice):
    session_folder_list = glob.glob(str(w_dir / f'JK{mouse:03d}S*'))
    for session_folder in tqdm(session_folder_list):
        session_name = session_folder.split('\\')[-1]
        save_fn = save_dir / f'{session_name}_touch_whisker_features.pkl'
        # if save_fn.exists():
        #     continue
        wf_fn = w_dir / session_name / f'{session_name}_whisker_final_h5.mat'
        b_fn = b_dir / f'JK{mouse:03d}' / f'{session_name}_behavior.pkl'
        if not wf_fn.exists() or not b_fn.exists():
            continue
        whisker_pkl_fn = w_dir / session_name / f'{session_name}_whisker.pkl'
        whisker_df = pd.read_pickle(whisker_pkl_fn)
        b_df = pd.read_pickle(b_fn)
        touch_features = get_session_touch_whisker_features(whisker_df, b_df)
        touch_features.to_pickle(save_fn)


  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

...

  0%|          | 0/31 [00:00<?, ?it/s]

  0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

# Debugging

In [125]:
trialNums = whisker_df.index.values
touch_features = {}
for trialNum in trialNums:
    if len(touch_features) == 0:
        temp_features = get_touch_feature_df_each_trial(whisker_df.loc[trialNum], b_df.loc[trialNum])
        if temp_features is not None:
            touch_features = temp_features
    else:
        temp_features = get_touch_feature_df_each_trial(whisker_df.loc[trialNum], b_df.loc[trialNum])
        if temp_features is not None:
            touch_features = pd.concat([touch_features, temp_features], ignore_index=True)